In [1]:
!conda install -q -y gdown

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.10.0      |     pyha770c72_0          77 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_2         145 KB  conda-forge
    conda-4.12.0               |   py37h89c1867_0         1.0 MB  conda-forge
    filelock-3.6.0             |     pyhd8ed1ab_0          12 KB  conda-forge
    gdown-4.4.0                |     pyhd8ed1ab_0          16 KB  conda-forge
    openssl-1.1.1n             |       h166bdaf_0         2.1 MB  conda-forge
    soupsieve-2.3.1            |     pyhd8ed1ab_0          33 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLE

# Downloading and Uzipping Data

In [2]:
!gdown http://drive.google.com/uc?id=1B_UZtU4W65ZViTJsLeFfvK-xXCYUhw2A
!unzip -q dataset.zip

Downloading...
From: http://drive.google.com/uc?id=1B_UZtU4W65ZViTJsLeFfvK-xXCYUhw2A
To: /kaggle/working/dataset.zip
100%|██████████████████████████████████████| 1.13G/1.13G [00:16<00:00, 67.6MB/s]


# Importing the libraries

In [3]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import math
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim
from transformers import get_cosine_schedule_with_warmup
import warnings
warnings.filterwarnings('ignore')
from sklearn import  model_selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setting seed for reproducibility

In [4]:
def set_seed(seed):
    #Sets the seed for Reprocudibility
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
set_seed(42)

# Preparing the data before feeding to Model

In [5]:
train_path1 = './dataset/train/no' #path to training images with no substructure
train_path2 = './dataset/train/sphere' #path to training images with spherical substructure
train_path3 = './dataset/train/vort' #path to training images with vortex substructure
val_path1 = './dataset/val/no' #path to validation images with no substructure
val_path2 = './dataset/val/sphere' #path to validation images with spherical substructure
val_path3 = './dataset/val/vort' #path to validation images with vortex substructure

In [6]:
# function to return a numpy array with shape (no. of images x 2)
# the first column contains the image paths
# the second column contains the target
def make_data(path,class_name):
    if(class_name == 'no'):
        ss = 1
    elif(class_name == 'sphere'):
        ss = 0
    else:
        ss = -1
    #no substructure -> 0 (target)
    #spherical substructure -> 1 (target)
    #vortex substructure -> 2 (target)

    pth = np.array(os.listdir(path))
    pth = ['./dataset/train/' + f'{class_name}/'+ i for  i in pth]
    pth = np.array(pth)
    pth = np.expand_dims(pth,1)
    # if ss = 1, label = 0
    # if ss = 0, label = 1
    # if ss = -1, label = 2
    label = np.ones((len(pth),1)) - ss  
    data = np.concatenate([pth,label],axis = 1)
    return data

In [7]:
d1 = make_data(train_path1 , 'no')
d2 = make_data(train_path2 , 'sphere')
d3 = make_data(train_path3 , 'vort')
v1 = make_data(val_path1 , 'no')
v2 = make_data(val_path2 , 'sphere')
v3 = make_data(val_path3 , 'vort')

In [8]:
X_val = np.concatenate([v1,v2,v3] , axis = 0)
t_data = np.concatenate([d1,d2,d3] , axis = 0)

# Splitting into train and test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test = train_test_split(t_data, test_size=0.1, random_state=42) # 10% set to test data

In [11]:
print(X_train.shape) # training data

(27000, 2)


In [12]:
print(X_test.shape) # test data

(3000, 2)


In [13]:
print(X_val.shape) # val data

(7500, 2)


# Best Hyperparameters found

In [14]:
image_size = 150
smoothing_factor = 0.1 #for loss function
lr=3e-4
weight_decay = 1e-5
epochs= 20
warmup_epochs = 3

# The Model

This is a custom model coded by me which utilizes the idea of Densenet(https://arxiv.org/abs/1608.06993) and Convolutional Block Attention Module(CBAM)(https://arxiv.org/abs/1807.06521).


The Model contains  "Neck" too ,which is made of (Linear -> Relu -> BatchNorm ->Linear -> BatchNorm -> final layer)

In [15]:
class Dense_layer(nn.Module):
    def __init__(self,in_chans):
        super().__init__()
        self.net = nn.Sequential(
            nn.BatchNorm2d(in_chans),
            nn.PReLU(),
            nn.Conv2d(in_chans, in_chans, kernel_size=1, padding=1, bias=False),
            nn.BatchNorm2d(in_chans),
            nn.PReLU(),
            nn.Conv2d(in_chans, in_chans*2, kernel_size=3, bias=False)
        )
    def forward(self,x):
        return self.net(x)

The Denseblock contains 2 Denselayers.
Each Denselayer is composed of 2 Convolutional layers along with batchnorm.
One thing I have changed from original paper is the activation function. They used ReLu whereas I use Parametric ReLu(PreLu). 
Prelu was giving better results

In [16]:
class Dense_block(nn.Module):
    def __init__(self,in_chans):
        super().__init__()
        self.d1 = Dense_layer(in_chans)
        self.d2 = Dense_layer(in_chans+in_chans*2)
    def forward(self,x):
        op1 = self.d1(x)
        op1 = torch.cat([op1,x],dim =1)
        op2 = self.d2(op1)
        return op2

In [17]:
class Channel_A_Block(nn.Module):
    def __init__(self,in_chans):
        super().__init__()
        self.n = in_chans
        self.maxpool = nn.AdaptiveMaxPool2d((1))
        self.avgpool = nn.AdaptiveAvgPool2d((1))
        self.MLP = nn.Sequential(
            nn.Linear(self.n ,64 ,bias=False),
            nn.PReLU(),
            nn.Linear( 64, self.n,bias=False)
        )
    def forward(self,x):
        x_m = self.maxpool(x)
        x_m = x_m.view(x_m.shape[0],-1)
        x_a = self.avgpool(x)
        x_a = x_a.view(x_a.shape[0],-1)

        x_m = self.MLP(x_m)
        x_a = self.MLP(x_a)

        out = x_m + x_a
        out = out.view(out.shape[0],self.n,1,1)
        out = F.sigmoid(out)

        return out

In [18]:
class Spatial_A_Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.maxp = nn.AdaptiveMaxPool3d((1,None,None))
        self.avgp = nn.AdaptiveMaxPool3d((1,None,None))
        self.conv = nn.Conv2d(2 , 1 , 7, padding=7//2, bias=False)
     
    def forward(self,x):
        x_m = self.maxp(x)
        x_a = self.avgp(x)
        op = torch.cat([x_m , x_a] , dim =1)
        op = F.sigmoid(self.conv(op))
        return op

The Attention block is made of channel attention block and spatial attention block.

In [19]:
class A_Block(nn.Module):
    def __init__(self,in_chans):
        super().__init__()
        self.c_a = Channel_A_Block(in_chans)
        self.s_a = Spatial_A_Block()

    def forward(self,x):
        Fc = self.c_a(x)
        Fc = Fc*x
        Fm = self.s_a(Fc)
        F = Fm*Fc
        return F

The idea of transition Layer is given in the densenet paper, where it reduces the spatial length & width of the incoming feature.

In [20]:
class TransitionLayer(nn.Module):

    def __init__(self, c_in, c_out):
        super().__init__()
        self.transition = nn.Sequential(
            nn.BatchNorm2d(c_in),
            nn.PReLU(),
            nn.Conv2d(c_in, c_out, kernel_size=1,stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2) # Average the output for each 2x2 pixel group
        )

    def forward(self, x):
        return self.transition(x)

The full model architecture is given as


![](https://i.imgur.com/7iAJB4i.png)

In [21]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 =  nn.Conv2d(1,4, kernel_size=3)
        
        self.dense_block1 = Dense_block(4)
        self.a_b1 = A_Block(24) #inchans = 2*(prev_chans + 2*prev_chans) hence (2*(4 + 2*4) = 24)
        self.tras1 = TransitionLayer(24 , 32)
        
        self.dense_block2 = Dense_block(32)
        self.a_b2 = A_Block(192)
        self.tras2 = TransitionLayer(192 , 360)
        
        self.pool = nn.AdaptiveAvgPool2d((1))
     
        self.neck = nn.Sequential(
            nn.Linear(360,360,bias =False),
            nn.ReLU(),
            nn.BatchNorm1d(360),
            nn.Linear(360 , 64,bias =False),
            nn.BatchNorm1d(64),
            nn.Linear(64,3)
        )
        
    def forward(self,x):
        x = self.conv1(x)
        #dense_attention block 1
        x = self.dense_block1(x)
        x = self.a_b1(x)
        x = self.tras1(x)

        #dense_attention block2
        x = self.dense_block2(x)
        x = self.a_b2(x)
        x = self.tras2(x)
    
        x = self.pool(x)
        x = x.view(x.shape[0],-1)
        
        x = self.neck(x)
        return x

# Augmentations

In [22]:
#In my experiments I saw no augmentations was giving the best result
train_aug = A.Compose(
        [  
        A.Resize(image_size,image_size,p=1.0),    
        ToTensorV2()
        ]
        )


#VALIDATING AUGMENTATIONS
val_aug = A.Compose(
        [ 
        A.Resize(image_size,image_size,p=1.0),
        ToTensorV2()
        ]
        )

# Initializing the Dataset

In [23]:
class Len(Dataset):
    def __init__(self , data , augs):
        self.data = data
        self.augs = augs
        
    def __len__(self):
        return(len(self.data))
    
    def __getitem__(self , idx):
        
        img_src = self.data[idx][0]
        
        image = np.load(img_src)
        image = image - 0.5
        image = np.squeeze(image, axis = 0)
        transformed = self.augs(image=image)       
        image = transformed['image']
        image = torch.tensor(image,dtype = torch.float32) 

        target = torch.tensor(float(self.data[idx][1]))
    
        return image,torch.tensor(target).long() 

# Helper Functions

In [24]:
class AverageMeter(object):
    #Computes and stores the average and current value
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [25]:
def train_one_epoch(train_loader,model,optimizer,criterion,e,epochs,scheduler):
    losses = AverageMeter()
    scores = AverageMeter()
    scoresx = AverageMeter()
    model.train()
    global_step = 0
    loop = tqdm(enumerate(train_loader),total = len(train_loader))
    
    for step,(image,labels) in loop:
        image = image.to(device)
        labels= labels.to(device)
        output2 = model(image)
        batch_size = labels.size(0)
        
        loss = criterion(output2,labels)
        
        out = output2.softmax(1)
        outputs = torch.argmax(out, dim=1).cpu().detach().numpy()
        targets = labels.cpu().detach().numpy()
        f1 = f1_score(targets, outputs , average='macro')
        acc = accuracy_score(targets, outputs)
        losses.update(loss.item(), batch_size)
        scores.update(f1.item(), batch_size)
        scoresx.update(acc.item(), batch_size)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
        global_step += 1
        
        loop.set_description(f"Epoch {e+1}/{epochs}")
        loop.set_postfix(loss = loss.item(), f1= f1.item(),acc= acc.item(), stage = 'train')
        
        
    return losses.avg,scores.avg,scoresx.avg

In [26]:
def val_one_epoch(loader,model,criterion):
    losses = AverageMeter()
    scores = AverageMeter()
    scoresx = AverageMeter()
    model.eval()
    global_step = 0
    loop = tqdm(enumerate(loader),total = len(loader))
    
    for step,(image, labels) in loop:
        image = image.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            output2 = model(image)
    
        loss = criterion(output2,labels)
      
        out = output2.softmax(1)
        outputs = torch.argmax(out, dim=1).cpu().detach().numpy()
        targets = labels.cpu().detach().numpy()
        f1 = f1_score(targets, outputs , average='macro')
        acc = accuracy_score(targets, outputs)
        losses.update(loss.item(), batch_size)
        scores.update(f1.item(), batch_size)
        scoresx.update(acc.item(), batch_size)
        loop.set_postfix(loss = loss.item(), f1= f1.item(),acc=acc.item(), stage = 'valid')
        
    return losses.avg,scores.avg,scoresx.avg

In [27]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Loss function

Cross_Entropy was doing fine, but it was overfiiting. 
So I read about Label Smoothing BCE (from here https://towardsdatascience.com/what-is-label-smoothing-108debd7ef06)
What it is doing is that, it is decreasing the amount of overconfident predictions(very close to 1 or 0). Which maybe responsible for overfitting, the loss function here works as a regularizer.

# Training trick
For the first few epochs(here 3 epochs) CrossEntropy Loss is used to make the model confident.
After that Label_Smoothing loss is used.
After many experimentations I have found this gives the best performing model

In [28]:
#Implementation from fastai https://github.com/fastai/fastai2/blob/master/fastai2/layers.py#L338
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, ε:float=smoothing_factor, reduction='mean'):
        super().__init__()
        self.ε,self.reduction = ε,reduction
    
    def forward(self, output, target):
        # number of classes
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        # (1-ε)* H(q,p) + ε*H(u,p)
        return (1-self.ε)*nll + self.ε*(loss/c) 

# Training Loop

In [29]:
def fit(m,training_batch_size=64,validation_batch_size=64):
    

    train_data= Len(X_train , augs = train_aug)
    val_data  = Len(X_val , augs = val_aug)
    
    train_loader = DataLoader(train_data,
                             shuffle=True,
                        num_workers=4,pin_memory=True, drop_last=True,
                        batch_size=training_batch_size)
    valid_loader = DataLoader(val_data,
                             shuffle=False,
                        num_workers=4,pin_memory=True, drop_last=False,
                        batch_size=validation_batch_size)
   
    #loss functions
    criterion1= LabelSmoothingCrossEntropy()
    criterion2= nn.CrossEntropyLoss()
    #optimizer
    optimizer = optim.AdamW(m.parameters(), lr=lr, weight_decay = weight_decay)
    num_train_steps = math.ceil(len(train_loader))
    num_warmup_steps= num_train_steps * warmup_epochs
    num_training_steps=int(num_train_steps * epochs)
    
    #learning rate scheduler
    sch = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps = num_warmup_steps,num_training_steps =num_training_steps) 

    loop = range(epochs)
    for e in loop:
        
        # training trick
        if(e< warmup_epochs):
            criterion = criterion2
        else:
            criterion = criterion1
            
        train_loss,train_f1,t_acc = train_one_epoch(train_loader,m,optimizer,criterion,e,epochs,sch)
    
        print(f'For epoch {e+1}/{epochs}')
        print(f'average train_loss {train_loss}')
        print(f'average train_f1 {train_f1}' )
        print(f'average train_acc {t_acc}' )
        
        val_loss,val_f1,v_acc= val_one_epoch(valid_loader,m,criterion)
        
        print(f'avarage val_loss { val_loss }')
        print(f'avarage val_f1 {val_f1}')
        print(f'avarage val_acc {v_acc}')

        torch.save(m.state_dict(),OUTPUT_DIR+ f' val_acc {v_acc}.pth')

In [30]:
mod = Model()
mod.to(device)
fit(mod)

  0%|          | 0/421 [00:00<?, ?it/s]

For epoch 1/20
average train_loss 1.1239563478709966
average train_f1 0.3272520349180383
average train_acc 0.34623663895486934


  0%|          | 0/118 [00:00<?, ?it/s]

avarage val_loss 1.1078339441299438
avarage val_f1 0.1685418281434257
avarage val_acc 0.346


  0%|          | 0/421 [00:00<?, ?it/s]

...

avarage val_loss 0.3639391976515452
avarage val_f1 0.5309688674054328
avarage val_acc 0.9674666666666667


  0%|          | 0/421 [00:00<?, ?it/s]

For epoch 20/20
average train_loss 0.3534559613876841
average train_f1 0.9715814434717734
average train_acc 0.9726098574821853


  0%|          | 0/118 [00:00<?, ?it/s]

avarage val_loss 0.3646840662479401
avarage val_f1 0.5290248296269892
avarage val_acc 0.9688


In [31]:
#delete the data after training
import shutil
shutil.rmtree("./dataset/")